# Integrating CrewAI Tools with the AG2 Framework

In this tutorial, we demonstrate how to integrate [CrewAI Tools](https://github.com/crewAIInc/crewAI-tools/tree/main) into the AG2 framework. This process enables smooth interoperability between the two systems, allowing developers to leverage CrewAI's powerful tools within AG2's flexible agent-based architecture. By the end of this guide, you will understand how to configure agents, convert CrewAI tools for use in AG2, and validate the integration with a practical example.


## Installation
Install the required packages for integrating CrewAI tools into the AG2 framework.
This ensures all dependencies for both frameworks are installed.

```bash
pip install ag2[interop-crewai]
```

## Imports

Import necessary modules and tools.
- `os` is used to access environment variables.
- `Path` helps in handling file paths.
- `TemporaryDirectory` is used for creating a temporary workspace.
- `FileWriterTool` and `ScrapeWebsiteTool` are the CrewAI tools we will integrate.
- `AssistantAgent` and `UserProxyAgent` are core AG2 classes.
- `CrewAIInteroperability` facilitates the interoperability between AG2 and CrewAI.

In [ ]:
import os
from pathlib import Path
from tempfile import TemporaryDirectory

from crewai_tools import FileWriterTool, ScrapeWebsiteTool

from autogen import AssistantAgent, UserProxyAgent
from autogen.interoperability.crewai import CrewAIInteroperability

## Agent Configuration

Configure the agents for the interaction.
- `config_list` defines the LLM configurations, including the model and API key.
- `UserProxyAgent` simulates user inputs without requiring actual human interaction (set to `NEVER`).
- `AssistantAgent` represents the AI agent, configured with the LLM settings.

In [2]:
config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
)

chatbot = AssistantAgent(
    name="chatbot",
    llm_config={"config_list": config_list},
)

## Tool Integration

Initialize and register the CrewAI tool with AG2.
- `crewai_tool` is an instance of the `FileWriterTool` from CrewAI.
- `CrewAIInteroperability` converts the CrewAI tool to make it usable in AG2.
- `register_for_execution` and `register_for_llm` allow the tool to work with the UserProxyAgent and AssistantAgent.

In [3]:
crewai_tool = FileWriterTool()
crewai_interop = CrewAIInteroperability()
ag2_tool = crewai_interop.convert_tool(crewai_tool)

ag2_tool.register_for_execution(user_proxy)
ag2_tool.register_for_llm(chatbot)

## File creation

Demonstrate the integration by writing to a file using the converted CrewAI tool.
- A temporary directory is created to simulate a file operation environment.
- The `message` instructs the chatbot to use the tool to write a specific string into a file.
- `user_proxy.initiate_chat` starts the interaction, with the chatbot processing the request and using the tool.
- Finally, the output file is verified to ensure the integration works correctly.

In [ ]:
with TemporaryDirectory() as tmpdirname:
    filename = "tool_result.txt"
    message = f"""Write 'Easy Migration :)' into {filename}.
Use {tmpdirname} dir.
"""

    user_proxy.initiate_chat(recipient=chatbot, message=message, max_turns=2)

    assert Path(tmpdirname, filename).read_text() == "Easy Migration :)"

In [ ]:
crewai_scrape_tool = ScrapeWebsiteTool()
ag2_tool = crewai_interop.convert_tool(crewai_scrape_tool)

ag2_tool.register_for_execution(user_proxy)
ag2_tool.register_for_llm(chatbot)

message = "Scape the website https://ag2.ai/"

chat_result = user_proxy.initiate_chat(recipient=chatbot, message=message, max_turns=2)

In [ ]:
print(chat_result.summary)